---
# --- Day 20: Jurassic Jigsaw ---
---

In [1]:
import numpy as np

### Input

In [2]:
with open("data/20_input.txt") as f:
    data = [l.strip() for l in f.readlines()]

In [8]:
def interpret_data(data):
    frames = {}    
    new_tile = True
    i_start = None
    for i, l in enumerate(data):
        last_row = i == (len(data) - 1)
        if new_tile:
            key = int(l[5:9])
            i_start = i+1
            new_tile = False
        elif (l == "") or last_row:
            m = np.matrix([[int(p=="#") for p in row] for row in data[i_start:i+int(last_row)]])
            frames[key] = m
            new_tile = True
    return frames

In [9]:
frames = interpret_data(data)

In [10]:
len(frames)

144

### Part 1: arrange frames

In [43]:
def get_edges(mat, flip=False):
    h, w = mat.shape
    edges = [np.array(mat[0,:]).flatten(), 
             np.array(mat[h-1,:]).flatten(), 
             np.array(mat[:,0]).flatten(),
             np.array(mat[:,w-1]).flatten()
            ]
    if flip:
        flipped_edges = [np.flip(e) for e in edges]
        edges = np.vstack([edges, flipped_edges])
    return edges

In [132]:
edges = {i: get_edges(mat) for i, mat in frames.items()}

In [59]:
def get_matching_edges(frames, frame_idx, edges):
    frame_edges = get_edges(frames[frame_idx], flip=True)
    other_edges = np.vstack([v for key, v in edges.items() if key != frame_idx])
    return [np.any([np.allclose(e_test, e_ref) for e_ref in other_edges]) for e_test in frame_edges]       

In [60]:
for k in frames.keys():
    matching_list  = get_matching_edges(frames, k, edges)
    matching_edges = sum(matching_list)
    if matching_edges <= 2:
        print(f"Title {k}, matching edges={matching_edges}.")

Title 3607, matching edges=2.
Title 1697, matching edges=2.
Title 2731, matching edges=2.
Title 1399, matching edges=2.


In [46]:
3607*1697*2731*1399

23386616781851

### Part 2: find sea monster

#### Place one tile at the top left corner

In [270]:
def get_all_matrix_versions(m):
    return [m, np.fliplr(m),
            np.rot90(m, -1), np.fliplr(np.rot90(m, -1)),
            np.rot90(m, -2), np.fliplr(np.rot90(m, -2)),
            np.rot90(m, -3), np.fliplr(np.rot90(m, -3))                   
           ]

In [271]:
def check_edge_couple(e1, e2):
    e_versions = [e1, np.fliplr(e1), np.vstack([e1[0,:], np.flip(e1[1,:])]), np.vstack([np.flip(e1[0,:]), e1[1,:]])]
    #e_versions = [e1]
    match = False
    for e in e_versions:
        match = match | np.allclose(e.flatten(), e2.flatten()) | np.allclose(np.flipud(e).flatten(), e2.flatten())
    return match

In [272]:
def find_corner_position(idx, frames, edges):
    frame_edges = get_edges(frames[idx], flip=True)
    edges_to_match = frame_edges[get_matching_edges(frames, idx, edges)]
    all_versions = get_all_matrix_versions(frames[idx])
    # top left ?
    tl_match = [check_edge_couple(edges_to_match, np.array([v[-1,:].flatten(), v[:,-1].flatten()])) for v in all_versions]
    if np.any(tl_match):
        print(f"top left is possible, versions of the matrix: {np.where(tl_match)[0]}")
    # bottom left ?
    tl_match = [check_edge_couple(edges_to_match, np.array([v[0,:].flatten(), v[:,-1].flatten()])) for v in all_versions]
    if np.any(tl_match):
        print(f"bottom left is possible, versions of the matrix: {np.where(tl_match)[0]}")
    # top right ?
    tl_match = [check_edge_couple(edges_to_match, np.array([v[-1,:].flatten(), v[:,0].flatten()])) for v in all_versions]
    if np.any(tl_match):
        print(f"top right is possible, versions of the matrix: {np.where(tl_match)[0]}")
    # bottom right?
    tl_match = [check_edge_couple(edges_to_match, np.array([v[0,:].flatten(), v[:,0].flatten()])) for v in all_versions]
    if np.any(tl_match):
        print(f"bottom right is possible, versions of the matrix: {np.where(tl_match)[0]}")

In [273]:
find_corner_position(3607, frames, edges)

top left is possible, versions of the matrix: [0 3]
bottom left is possible, versions of the matrix: [5 6]
top right is possible, versions of the matrix: [1 2]
bottom right is possible, versions of the matrix: [4 7]


In [274]:
find_corner_position(1697, frames, edges)

top left is possible, versions of the matrix: [0 3]
bottom left is possible, versions of the matrix: [5 6]
top right is possible, versions of the matrix: [1 2]
bottom right is possible, versions of the matrix: [4 7]


In [275]:
find_corner_position(2731, frames, edges)

top left is possible, versions of the matrix: [0 3]
bottom left is possible, versions of the matrix: [5 6]
top right is possible, versions of the matrix: [1 2]
bottom right is possible, versions of the matrix: [4 7]


In [276]:
find_corner_position(1399, frames, edges)

top left is possible, versions of the matrix: [0 3]
bottom left is possible, versions of the matrix: [5 6]
top right is possible, versions of the matrix: [1 2]
bottom right is possible, versions of the matrix: [4 7]


In [277]:
reconstructed_image[0, 0, :] = frames[3607]
filled[0, 0] = True
del remaining_frames[3607]

#### Choosing other images

In [268]:
reconstructed_image = np.zeros((12, 12, 10, 10))
filled = np.zeros((12, 12), dtype=bool)

In [269]:
remaining_frames = frames.copy()

In [278]:
def check_mask_match(m, mask):
    return np.allclose(m[mask!=-1], mask[mask!=-1])

In [279]:
fsize = 10
m, n = filled.shape
stuck = False
i = 0
while not (np.all(filled.flatten()) or stuck):
    i += 1
    print(f"--- ITERATION # {i} ---")
    cnt = 0
    for i in range(m):
        for j in range(n):
            if filled[i, j]:
                print(f"({i}, {j}) position already ok.")
                pass
            else:
                mask = np.matrix(-1*np.ones((fsize, fsize)))
                if (i > 0) and filled[i-1, j]:
                    mask[0, :] = reconstructed_image[i-1, j, -1, :].reshape((1, fsize))
                if (i < (m - 1)) and filled[i+1, j]:
                    mask[-1, :] = reconstructed_image[i+1, j, 0, :].reshape((1, fsize))
                if (j > 0) and filled[i, j-1]:
                    mask[:, 0] = reconstructed_image[i, j-1, :, -1].reshape((fsize, 1))
                if (j < (n - 1)) and filled[i, j+1]:
                    mask[:, -1] = reconstructed_image[i, j+1, :, 0].reshape((fsize, 1))
                matching_ids = []
                matching_images = []
                for k, f in remaining_frames.items():
                    all_frames = get_all_matrix_versions(f)
                    matches = [check_mask_match(im, mask) for im in all_frames]
                    if sum(matches)==1:
                        matching_images.append(all_frames[np.where(matches)[0][0]])
                        matching_ids.append(k)
                if len(matching_ids) == 1:
                    print(f"Exactly one good match found for position ({i}, {j}): tile <{matching_ids[0]}>.")
                    reconstructed_image[i, j, :] = matching_images[0]
                    filled[i, j] = True
                    del remaining_frames[matching_ids[0]]
                    cnt += 1
                elif len(matching_ids) > 1:
                    print(f"Too many candidates found for position ({i}, {j}).")
                else:
                    print(f"No good candidate found for position ({i}, {j}).")
    stuck = (cnt ==0)
print()
if stuck:
    print("Unfortunately I am stuck. :(")
else:
    print("All tiles have been placed!")

--- ITERATION # 1 ---
(0, 0) position already ok.
Exactly one good match found for position (0, 1): tile <1289>.
Exactly one good match found for position (0, 2): tile <3347>.
Exactly one good match found for position (0, 3): tile <2143>.
Exactly one good match found for position (0, 4): tile <1567>.
Exactly one good match found for position (0, 5): tile <3413>.
Exactly one good match found for position (0, 6): tile <3691>.
Exactly one good match found for position (0, 7): tile <3911>.
Exactly one good match found for position (0, 8): tile <2593>.
Exactly one good match found for position (0, 9): tile <2657>.
Exactly one good match found for position (0, 10): tile <3217>.
Exactly one good match found for position (0, 11): tile <1697>.
Exactly one good match found for position (1, 0): tile <1031>.
Exactly one good match found for position (1, 1): tile <2953>.
Exactly one good match found for position (1, 2): tile <2939>.
Exactly one good match found for position (1, 3): tile <1231>.
Exa

Exactly one good match found for position (11, 2): tile <3373>.
Exactly one good match found for position (11, 3): tile <3169>.
Exactly one good match found for position (11, 4): tile <3011>.
Exactly one good match found for position (11, 5): tile <2473>.
Exactly one good match found for position (11, 6): tile <3253>.
Exactly one good match found for position (11, 7): tile <2203>.
Exactly one good match found for position (11, 8): tile <3847>.
Exactly one good match found for position (11, 9): tile <1193>.
Exactly one good match found for position (11, 10): tile <1949>.
Exactly one good match found for position (11, 11): tile <2731>.

All tiles have been placed!


#### Combine all tiles

In [340]:
full_img = np.matrix(-1*np.ones((12*8, 12*8)))

In [341]:
for i in range(reconstructed_image.shape[0]):
    for j in range(reconstructed_image.shape[1]):
        full_img[i*8:(i+1)*8, j*8:(j+1)*8] = reconstructed_image[i, j, 1:-1, 1:-1]

#### Find sea monster

In [343]:
with open("data/20_sea_monster.txt") as f:
    data_sm = [l.strip() for l in f.readlines()]
with open("data/20_image_test.txt") as f:
    data_test = [l.strip() for l in f.readlines()]

In [344]:
sm = np.matrix([[int(p=="#") for p in row] for row in data_sm])
sm[sm==0] = -1
full_img_test = np.matrix([[int(p=="#") for p in row] for row in data_test])

In [345]:
def find_sea_monster_in_image(img, sm):
    h, w = img.shape
    m, n = sm.shape
    good_pos = []
    for i in range(h-m+1):
        for j in range(w-n+1):
            if check_mask_match(img[i:i+m, j:j+n], sm):
                good_pos.append((i, j))
    return good_pos

In [346]:
all_images = get_all_matrix_versions(full_img)
best_pos = []
for i, img in enumerate(all_images):
    print(f"Image version #{i}:", end=" ")
    good_pos = find_sea_monster_in_image(img, sm)
    print(f"{len(good_pos)} monsters found.")
    if len(good_pos) > len(best_pos):
        best_pos = good_pos

Image version #0: 0 monsters found.
Image version #1: 0 monsters found.
Image version #2: 0 monsters found.
Image version #3: 19 monsters found.
Image version #4: 0 monsters found.
Image version #5: 0 monsters found.
Image version #6: 0 monsters found.
Image version #7: 0 monsters found.


In [347]:
best_img = all_images[3]

In [348]:
sm_neg = sm.copy()
sm_neg[sm_neg == 1] = 0
sm_neg[sm_neg == -1] = 1

In [349]:
best_pos

[(2, 17),
 (7, 69),
 (14, 57),
 (30, 55),
 (31, 6),
 (34, 33),
 (36, 64),
 (41, 66),
 (45, 23),
 (46, 52),
 (50, 67),
 (52, 1),
 (65, 64),
 (66, 8),
 (67, 42),
 (72, 1),
 (76, 60),
 (84, 57),
 (85, 4)]

In [350]:
for p in best_pos:
    best_img[p[0]:p[0]+sm.shape[0], p[1]:p[1]+sm.shape[1]] = np.multiply(best_img[p[0]:p[0]+sm.shape[0], p[1]:p[1]+sm.shape[1]], sm_neg)

In [351]:
np.sum(best_img)

2376.0